## ANN dMap h2t Takens latent space -> dMap all-atom latent space

In [1]:
import os
import matplotlib as mpl
if os.environ.get('DISPLAY','') == '':
    print('no display found. Using non-interactive Agg backend')
    mpl.use('Agg')
import matplotlib.pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import sys
import math
import time
import pickle
from copy import deepcopy
import numpy as np
import pandas as pd
import torch
from torch import nn #import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
import torch.utils.data
from torch.utils.data import Dataset, TensorDataset, DataLoader
import torch.nn.parallel
import torch.optim as optim
from torch.autograd import Variable
import torch.utils.data
from torch.utils.data.dataset import random_split
import sklearn.preprocessing as pre
from sklearn.model_selection import train_test_split
from collections import OrderedDict
import mdtraj as md
device = torch.device("cpu") #or GPU

In [2]:
showPlots=1
useMagics=1
if useMagics:
    %matplotlib inline
    #%matplotlib notebook
    %load_ext autoreload
    %autoreload 2'''

In [3]:
#define functions for ANN training
def make_train_step(model, loss_fn, optimizer):
    # Builds function that performs a step in the train loop
    def train_step(x, y):
        # Sets model to TRAIN mode
        model.train()
        # Makes predictions
        yhat = model(x)
        # Computes loss
        loss = loss_fn(y, yhat)
        # Computes gradients
        loss.backward()
        # Updates parameters and zeroes gradients
        optimizer.step()
        optimizer.zero_grad()
        # Returns the loss
        return loss.item()
    
    # Returns the function that will be called inside the train loop
    return train_step

def get_activation(name):
    def hook(model, input, output):
        activation[name] = output.detach()
    return hook

#npzfile = np.load('/home/maxtopel/Desktop/History/Gaussian Mol Rec/mol rec/Chignolin_partial/h2t.npz')
#sorted(npzfile.files)
#h2t = npzfile['h2t']

### All in


In [4]:
#Define noise, time delay and number of training points
tau = [1,5,10,25,50]
Dvec = [40000,20000,10000,1000]
Photon_No  = [0,1000000,100000,10000]
iii = 0

for Dv in Dvec:
    for t in tau:
        for Nv in Photon_No:
            
            npzfile = np.load('../0A_aa_h2t/h2t_%fns.npz' %(t*0.2)) 
            sorted(npzfile.files)
            h2t = npzfile['h2t_train']
    
            indx = []
            dim = 11
            j = 0
            # Train on Dv data points
            stopframe = Dv
            #Chose over training points
            totframe = int(round(len(h2t)*0.8))
            # number of partition
            nohist = 10
            #train 10 percent on each ile
            ratio = 1/nohist

            #find histogram values
            no = list(np.histogram(h2t[0:totframe],bins=nohist)[0])
            val = list(np.histogram(h2t[0:totframe],bins=nohist)[1][0:nohist])
            val.insert(0,0)
            #iterate over the number of histogram divisions
            for ile in range(nohist):
                #print(ile)
                # iterate over all frames
                for i in range(totframe):
                    #iterate over deciles
                    if j < stopframe*ratio*(ile+1):
                        # select that it is in the correct decile
                        if h2t[i] < val[ile+1] and h2t[i] > val[ile]:
                            for d in range(-dim+1,1):
                                if (i+d) not in indx:
                                    indx.append(i+d)
                                    j += 1

                    elif j >= stopframe:
                        break

            # loading z_aa
            npzfile = np.load('../1A_dMap_aa/data_%dk_%fns.npz' %(Dv,t*0.2))
           # sorted(npzfile.files)
            z_aa = npzfile['z']

            z_aa_features_idx = np.array([1,2])
            z_aa = z_aa[:,z_aa_features_idx]

            n_aa = z_aa.shape[0]
            d_aa = z_aa.shape[1]

            # scaling
            zscale_aa = pre.MinMaxScaler(feature_range=(0,1)) # changing feature range requires modification of chain rule in post-hoc gradient rescaling AND use of loss other than BCE

            Z_aa = zscale_aa.fit_transform(z_aa)

            # saving scaling associated with model
            D = {}
            D['zscale'] = zscale_aa
            file = open('ANN_z_aa_scaling_%dk_%dphoton_%fns.pkl' %(Dv,Nv,t*0.2), 'wb')
            pickle.dump(D, file)
            file.close()

            # loading z_Takens
            npzfile = np.load('../3A_dMap_h2t/data_%dk_%dphoton_%fns.npz' %(Dv,Nv,t*0.2))
            #sorted(npzfile.files)
            z_Takens = npzfile['z']

            z_Takens_features_idx = np.array([1,2])

            z_Takens = z_Takens[:,z_Takens_features_idx]

            n_Takens = z_Takens.shape[0]
            d_Takens = z_Takens.shape[1]

            zscale_Takens = pre.MinMaxScaler(feature_range=(0,1)) # changing feature range requires modification of chain rule in post-hoc gradient rescaling AND use of loss other than BCE

            Z_Takens = zscale_Takens.fit_transform(z_Takens)

            # saving scaling associated with model
            D = {}
            D['zscale'] = zscale_Takens
            file = open('ANN_z_Takens_scaling_%dk_%dphoton_%fns.pkl' %(Dv,Nv,t*0.2), 'wb')
            pickle.dump(D, file)
            file.close()

            z_aa = z_aa[:z_Takens.shape[0],:]
            Z_aa = Z_aa[:Z_Takens.shape[0],:]

            assert z_aa.shape[0] == Z_aa.shape[0]
            assert z_aa.shape[0] == z_Takens.shape[0]
            assert z_aa.shape[0] == Z_Takens.shape[0]

            class ANN(nn.Module):
                def __init__(self, q, d_in, d_out):
                    super(ANN, self).__init__()
                    self.hidden1 = nn.Linear(d_in, np.int(np.ceil(q*d_in)))
                    self.hidden2 = nn.Linear(np.int(np.ceil(q*d_in)), np.int(np.ceil(q*d_in)))
                    self.hidden3 = nn.Linear(np.int(np.ceil(q*d_in)), np.int(np.ceil(q*d_in)))
                    self.hidden4 = nn.Linear(np.int(np.ceil(q*d_in)), np.int(np.ceil(q*d_in)))
                    self.output = nn.Linear(np.int(np.ceil(q*d_in)), d_out)

                def forward(self, x):
                    x = torch.tanh(self.hidden1(x))
                    x = torch.tanh(self.hidden2(x))
                    x = torch.tanh(self.hidden3(x))
                    x = torch.tanh(self.hidden4(x))
                    x = torch.tanh(self.output(x))
                    return x

            activation = {}

            class CustomDataset(Dataset):
                def __init__(self, x_tensor, y_tensor):
                    self.x = x_tensor
                    self.y = y_tensor

                def __getitem__(self, index):
                    return (self.x[index], self.y[index])

                def __len__(self):
                    return len(self.x)

            x_tensor = torch.from_numpy(Z_Takens).float()
            y_tensor = torch.from_numpy(Z_aa).float()
            assert x_tensor.shape[0] == y_tensor.shape[0]

            idx = np.arange(x_tensor.shape[0])

            val_frac = 0.10
            idx_train, idx_val = train_test_split(idx, test_size=val_frac, random_state=42)

            train_dataset = CustomDataset(x_tensor[idx_train], y_tensor[idx_train])
            val_dataset = CustomDataset(x_tensor[idx_val], y_tensor[idx_val])

            q=5.

            model = ANN(q, d_Takens, d_aa).to(device)
            
            lr = 1e-4
            n_epochs = 100
            batch_size = 500

            loss_fn = nn.MSELoss(reduction='elementwise_mean')
            optimizer = torch.optim.Adam(model.parameters(), lr=lr, weight_decay=0)

            train_step = make_train_step(model, loss_fn, optimizer)

            n_batch = math.ceil(1. * len(train_dataset) / batch_size)

            loss_train = np.zeros(n_epochs)
            loss_val = np.zeros(n_epochs)

            start=time.time()

            for epoch in range(n_epochs):    

                dataset = DataLoader(dataset=train_dataset, batch_size=batch_size, shuffle=True, drop_last=True)

                model.train()

                batch_idx = 0
                for train_in, train_out in dataset:
                    train_in, train_out = train_in.to(device), train_out.to(device)
                    out = model(train_in)
                    loss = loss_fn(out, train_out) 
                    optimizer.zero_grad()
                    loss.backward() # scalar variables, when we call .backward() on them, don’t require arguments – only tensors require a matching sized tensor argument to be passed to the .backward() operation
                    optimizer.step()
                    batch_idx += 1

                model.eval()

                y_pred = model(train_dataset.x.to(device))
                diff = y_pred.cpu().detach().numpy() - train_dataset.y.cpu().detach().numpy()
                MSE = np.mean(diff**2)
                loss_train[epoch] = MSE

                y_pred = model(val_dataset.x.to(device))
                diff = y_pred.cpu().detach().numpy() - val_dataset.y.cpu().detach().numpy()
                MSE = np.mean(diff**2)
                loss_val[epoch] = MSE        

            end = time.time()

            save_path = "./ANN_%dk_%dphoton_%fns.pyt" %(Dv,Nv,t*0.2)
            torch.save(model.state_dict(), save_path)

            # INPUTS

            # loading z_Takens
            npzfile = np.load('../3A_dMap_h2t/data_%dk_%dphoton_%fns.npz' %(Dv,Nv,t*0.2))
           # sorted(npzfile.files)
            z_Takens = npzfile['z']

            # extracting features
            z_Takens = z_Takens[:,z_Takens_features_idx]

            # loading z_Takens scaling
            D = pickle.load(open("../4A_ANN_dMapaa2dMaph2t/ANN_z_Takens_scaling_%dk_%dphoton_%fns.pkl" %(Dv,Nv,t*0.2), "rb"))
            zscale_Takens = D['zscale']

            # scaling data
            Z_Takens = zscale_Takens.transform(z_Takens)

            # loading z_aa
            npzfile = np.load('../../Short Circuit/1A_dMap_aa/data_%dk_%fns.npz' %(Dv,t*0.2))
            #sorted(npzfile.files)
            z_aa = npzfile['z']

            # extracting features
            z_aa = z_aa[:,z_aa_features_idx]

            # loading z_aa scaling
            D = pickle.load(open("ANN_z_aa_scaling_%dk_%dphoton_%fns.pkl" %(Dv,Nv,t*0.2), "rb"))
            zscale_aa = D['zscale']

            # scaling data
            Z_aa = zscale_aa.transform(z_aa)

            # trimming data
            z_aa = z_aa[:Z_Takens.shape[0]]
            Z_aa = Z_aa[:Z_Takens.shape[0]]

            # initializing storage arrays
            Z_aa_pred = np.zeros(shape=(0,Z_aa.shape[1]), dtype=float)

            # running all data through model
            # N.B. running as batches so as not to max out GPU memory
            nSamples = Z_Takens.shape[0]
            maxBatchSz = 10000
            assert nSamples >= maxBatchSz
            nBatch = np.ceil(np.float(nSamples)/np.float(maxBatchSz)).astype(int)
            batchEdges = np.floor(np.linspace(0,nSamples,nBatch+1)).astype(int)

            #print('Running data through trained network...')
            for i in range(nBatch):
                data = torch.FloatTensor(Z_Takens[batchEdges[i]:batchEdges[i+1],:])
                data = Variable(data, requires_grad=False)
                data = data.to(device)
                model.eval()
                pred = model(data)
                Z_aa_pred_i = pred.cpu().detach().numpy()
                Z_aa_pred = np.append(Z_aa_pred,Z_aa_pred_i,axis=0)

            z_aa_pred = zscale_aa.inverse_transform(Z_aa_pred)

            # scaled MSE
            diff = Z_aa_pred - Z_aa
            MSE = np.mean(diff**2)
        
            # unscaled MSE
            diff = z_aa_pred - z_aa
            MSE = np.mean(diff**2)
           
            # saving latent space projections
            np.savez('ANN_pred_%dk_%dphoton_%fns.npz' %(Dv,Nv,t*0.2), z_aa=z_aa_pred)
           
            # initializing storage arrays
            Z_aa_pred = np.zeros(shape=(0,Z_aa.shape[1]), dtype=float)

            # running all data through model
            # N.B. running as batches so as not to max out GPU memory
            nSamples = Z_Takens.shape[0]
            maxBatchSz = 10000
            assert nSamples >= maxBatchSz
            nBatch = np.ceil(np.float(nSamples)/np.float(maxBatchSz)).astype(int)
            batchEdges = np.floor(np.linspace(0,nSamples,nBatch+1)).astype(int)

            #print('Running data through trained network...')
            for i in range(nBatch):
                data = torch.FloatTensor(Z_Takens[batchEdges[i]:batchEdges[i+1],:])
                data = Variable(data, requires_grad=False)
                data = data.to(device)
                model.eval()
                pred = model(data)
                Z_aa_pred_i = pred.cpu().detach().numpy()
                Z_aa_pred = np.append(Z_aa_pred,Z_aa_pred_i,axis=0)


            z_aa_pred = zscale_aa.inverse_transform(Z_aa_pred)

            # scaled MSE
            diff_ = Z_aa_pred - Z_aa
            s_MSE = np.mean(diff**2)
            s_RMSE = np.sqrt(s_MSE)

            # unscaled MSE
            diff = z_aa_pred - z_aa
            u_MSE = np.mean(diff**2)
            u_RMSE = np.sqrt(u_MSE)

            # saving latent space projections
            np.savez('ANN_pred_%dk_%dphoton_%fns.npz' %(Dv,Nv,t*0.2), z_aa=z_aa_pred)

            np.savez('MSE_%dk_%dphoton_%fns.npz' %(Dv,Nv,t*0.2), u_MSE=u_MSE, u_RMSE=u_RMSE, s_RMSE=s_RMSE, s_MSE=s_MSE )
            iii +=1
            print('\tProcessed model %d/%d' %(iii,len(tau)*len(Dvec)*len(Photon_No)))

/home/maxtopel/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:14: UserWarning: reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.
  warnings.warn("reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.")


	Processed model 1/80


/home/maxtopel/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:14: UserWarning: reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.
  warnings.warn("reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.")


	Processed model 2/80


/home/maxtopel/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:14: UserWarning: reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.
  warnings.warn("reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.")


...

	Processed model 78/80


/home/maxtopel/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:14: UserWarning: reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.
  warnings.warn("reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.")


	Processed model 79/80


/home/maxtopel/anaconda3/lib/python3.7/site-packages/torch/nn/_reduction.py:14: UserWarning: reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.
  warnings.warn("reduction='elementwise_mean' is deprecated, please use reduction='mean' instead.")


	Processed model 80/80
